In [3]:
from sklearn.datasets import load_iris
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import ClusteringEvaluator
import pandas as pd

spark = SparkSession.builder.appName('k-means').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/14 17:05:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/14 17:05:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
def load_data():
    iris = load_iris()
    X = iris.data
    y = iris.target
    feat_names = iris.feature_names
    
    df = pd.DataFrame(X, columns=feat_names)
    
    print(df.head(5))
    
    return spark.createDataFrame(df), y


def feature_assembler(spark_df, input_features):
    assembler = VectorAssembler(
        inputCols=input_features, outputCol='features')
    return assembler.transform(spark_df)


def k_means_clustering(spark_df, k=3):
    kmeans = KMeans().setK(k).setSeed(87)
    return kmeans.fit(spark_df)


def evaluate_model(model, spark_df):
    predictions = model.transform(spark_df)
    
    evaluator = ClusteringEvaluator()
    silhouette = evaluator.evaluate(predictions)
    print(f"Silhouette score: {silhouette}")
    
    return predictions
    

def cluster_counts(predictions):
    predictions.groupBy('prediction').count().show()
    



In [5]:
spark_df, y = load_data()

spark_df = feature_assembler(spark_df, spark_df.columns)

kmeans = k_means_clustering(spark_df, k=3)

predictions = evaluate_model(kmeans, spark_df)

cluster_counts(predictions)

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                5.1               3.5                1.4               0.2
1                4.9               3.0                1.4               0.2
2                4.7               3.2                1.3               0.2
3                4.6               3.1                1.5               0.2
4                5.0               3.6                1.4               0.2


24/11/14 17:06:00 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


Silhouette score: 0.735659605433223
+----------+-----+
|prediction|count|
+----------+-----+
|         1|   50|
|         2|   62|
|         0|   38|
+----------+-----+

